# Neural Networks in Reserving

### Initialization and Imports

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import os
import pandas as pd
import sys
import tensorflow as tf
import time

from pathlib import Path

# Add scripts path to PYTHONPATH
sys.path.append(Path(os.getcwd()).parent.__str__())

import _scripts.nn_reserve as nnr

# Constants
PATH = "/home/koscial/Downloads/data.txt"

### Reading the Data

In [3]:
df = nnr.read_data(PATH)

### Per-LoB triangles - comparison with the paper

In [12]:
tr_lob1 = nnr.compute_triangle(df[df.LoB == 1])
tr_lob1

,PayCum00,PayCum01,PayCum02,PayCum03,PayCum04,PayCum05,PayCum06,PayCum07,PayCum08,PayCum09,PayCum10,PayCum11,"C_i,J"
AY,,,,,,,,,,,,,
1994,71221308,107097616,119218852,126244729,130954687,134338201,136863858,138847649,140462706,141876332,143091824,144113215,144113215
1995,70445443,107509224,120188335,127743884,132657594,136172451,138911104,140982878,142667165,144185910,145509115,0,146557259
1996,70264354,106571489,119030089,126038984,130751283,134196341,136785783,138752321,140292123,141592049,0,0,143630044
1997,71391856,109126247,122420898,130046381,135191232,138888327,141706600,143848076,145562759,0,0,0,149369892
1998,72057031,110595627,124264547,132063848,137300200,141103281,143813256,145963881,0,0,0,0,150914478
1999,72950795,113367279,127872433,136097329,141500600,145428029,148286833,0,0,0,0,0,156477079
2000,74269641,117130929,133212405,142395739,148472485,152877799,0,0,0,0,0,0,165044594
2001,76854155,122357975,139487061,149374080,155805477,0,0,0,0,0,0,0,172723299
2002,78997419,124080898,140780532,150057915,0,0,0,0,0,0,0,0,171951785


In [14]:
compute_triangle(df[df.LoB == 2])

,PayCum00,PayCum01,PayCum02,PayCum03,PayCum04,PayCum05,PayCum06,PayCum07,PayCum08,PayCum09,PayCum10,PayCum11,"C_i,J"
AY,,,,,,,,,,,,,
1994,103185043,149888373,165808308,172908709,176408520,178204058,179898285,180594347,181497552,182224561,182607897,182382545,182382545
1995,97677453,143073597,159357282,166400870,170077965,172461990,173764385,174607136,175531292,176114088,176757351,0,176741285
1996,96740023,144915427,162571062,170338833,174144485,176105206,177460533,178241254,179251883,180220006,0,0,180691962
1997,89818493,133927673,151212605,157744627,161273026,163321828,164827473,166045800,166904486,0,0,0,168114113
1998,93975540,143100061,162325808,170802124,174793084,176884208,178352445,179455548,0,0,0,0,181252420
1999,94286447,146770768,167992960,177227679,181995131,185086124,187026748,0,0,0,0,0,191509813
2000,92626385,143523825,164246689,172858835,176570191,178901467,0,0,0,0,0,0,183875836
2001,95186192,147011576,166811817,175139254,178510745,0,0,0,0,0,0,0,185450670
2002,98384124,154441059,177514939,187150292,0,0,0,0,0,0,0,0,199824925


In [15]:
compute_triangle(df[df.LoB == 3])

,PayCum00,PayCum01,PayCum02,PayCum03,PayCum04,PayCum05,PayCum06,PayCum07,PayCum08,PayCum09,PayCum10,PayCum11,"C_i,J"
AY,,,,,,,,,,,,,
1994,86160180,135802273,151261411,158550269,162259500,165002200,166935852,168544942,169926114,170928839,171840530,172127503,172127503
1995,87022289,138472970,155540066,164270022,169169122,172687752,174834618,176722160,178297871,179589878,180590331,0,181188965
1996,79922069,130965741,149107330,157565863,161859469,164690349,166713433,167953578,168889009,169713858,0,0,171220927
1997,81397985,131988430,149900696,158366046,162827806,165688633,167727870,169458059,170936190,0,0,0,173783107
1998,81602766,130741959,148892452,157416734,162279161,165309761,167240986,168522372,0,0,0,0,171551281
1999,79981661,128718844,145734271,153160680,157342082,159519111,161167489,0,0,0,0,0,165251356
2000,79182494,128516831,144715443,151751631,155429791,157678683,0,0,0,0,0,0,163371271
2001,85884211,141964598,161695863,171307059,175945336,0,0,0,0,0,0,0,185481728
2002,84964194,138578696,156269563,163876551,0,0,0,0,0,0,0,0,176970009


In [16]:
compute_triangle(df[df.LoB == 4])

,PayCum00,PayCum01,PayCum02,PayCum03,PayCum04,PayCum05,PayCum06,PayCum07,PayCum08,PayCum09,PayCum10,PayCum11,"C_i,J"
AY,,,,,,,,,,,,,
1994,80107374,128927206,147331397,157995567,165309174,170554774,174693418,178071632,180814300,183238438,185370369,187132943,187132943
1995,82390307,133817406,153700031,165422644,173291819,179119490,183841165,187444360,190483967,193179250,195353355,0,197283895
1996,80764623,133382434,154398856,166669909,175122734,181007587,185762719,189488339,192504751,195207757,0,0,199484548
1997,78217202,128122921,148257372,159902546,167468603,172996153,177085488,180407413,182975460,0,0,0,188728027
1998,80514240,133721412,155515156,168181710,176723265,182969702,187681207,191525325,0,0,0,0,200991431
1999,83356775,139697248,163357170,177345268,186792644,193562758,198746016,0,0,0,0,0,213019408
2000,85730379,144106714,168660637,183168614,193274335,200506576,0,0,0,0,0,0,220893606
2001,87360045,147790884,173496302,188509617,198373810,0,0,0,0,0,0,0,225304878
2002,91378077,156182546,184522366,200739247,0,0,0,0,0,0,0,0,240324908


The figures are generally similar to those presented in the paper.

### Training Neural Networks

In [4]:
df.head()

,LoB,cc,AY,AQ,age,inj_part,RepDel,Pay00,Pay01,Pay02,...,PayCum02,PayCum03,PayCum04,PayCum05,PayCum06,PayCum07,PayCum08,PayCum09,PayCum10,PayCum11
0,1,14,1994,4,38,30,0,698,0,0,...,698,698,698,698,698,698,698,698,698,698
1,4,49,1994,1,31,34,0,616,0,0,...,616,616,616,616,616,616,616,616,616,616
2,4,1,1994,4,39,53,0,0,461,0,...,461,461,461,461,461,461,461,461,461,461
3,4,13,1994,4,32,53,0,0,234,0,...,234,234,234,234,234,234,234,234,234,234
4,4,45,1994,4,29,55,0,0,170,0,...,170,170,170,170,170,170,170,170,170,170


In [8]:
(df["PayCum00"] > 0) & (df.AY + df.RepDel <= df.AY.max())

0           True
1           True
2          False
3          False
4          False
           ...  
5003199     True
5003200     True
5003201     True
5003202    False
5003203     True
Length: 5003204, dtype: bool

In [ ]:
(df[f"PayCum{dev_year:02}"] > 0)
            & (df.AY + df.RepDel <= df.AY.max())

In [ ]:
df_preproc = nnr.preprocess_data(df)

models = []
times = []
for dev_year in range(12):
    tick = time.time()
    current_model = nnr.fit_model_nonzero(df_preproc, dev_year, 20, 100, 10000)
    models.append(current_model)
    tock = time.time() - tick
    times.append(tock)

In [17]:
a = time.time()

In [18]:
time.time() - a

6.225138187408447